In [ ]:
import sqlite3
import pandas as pd

# Connect to database
conn = sqlite3.connect("robot.db")
cur = conn.cursor()

# ---------- Drop old tables ----------
cur.execute("DROP TABLE IF EXISTS Robot;")
cur.execute("DROP TABLE IF EXISTS SensorReading;")
cur.execute("DROP TABLE IF EXISTS TargetInterval;")

# ---------- Create tables ----------
cur.execute("""
CREATE TABLE Robot(
    robot_id INTEGER PRIMARY KEY,
    name TEXT
);
""")

cur.execute("""
CREATE TABLE SensorReading(
    reading_id INTEGER PRIMARY KEY AUTOINCREMENT,
    robot_id INTEGER,
    timestamp INTEGER,
    x_coord REAL,
    y_coord REAL,
    FOREIGN KEY (robot_id) REFERENCES Robot(robot_id)
);
""")

cur.execute("""
CREATE TABLE TargetInterval(
    interval_id INTEGER PRIMARY KEY AUTOINCREMENT,
    start_time INTEGER,
    end_time INTEGER,
    event_type TEXT
);
""")

# ---------- Import robot.csv ----------
robots = pd.read_csv("csv_files/robot.csv")
robots.to_sql("Robot", conn, if_exists="append", index=False)

# ---------- Import t1.csv ... t5.csv ----------
for i in range(1, 6):
    df = pd.read_csv(f"csv_files/t{i}.csv")

    # CASE: t-files have only x and y, no timestamp
    if df.shape[1] == 2:
        df.columns = ["x_coord", "y_coord"]
        df["timestamp"] = range(1, len(df)+1)

    # CASE: t-files have timestamp, x, y
    elif df.shape[1] == 3:
        df.columns = ["timestamp", "x_coord", "y_coord"]

    else:
        raise ValueError(f"Unexpected column count in t{i}.csv: {df.columns}")

    df["robot_id"] = i

    df.to_sql("SensorReading", conn, if_exists="append", index=False)

# ---------- Import interval.csv ----------
ti = pd.read_csv("csv_files/interval.csv", header=None)
ti.columns = ["start_time", "end_time", "event_type"]


# Automatically rename to expected schema
ti.columns = [c.strip().lower() for c in ti.columns]

rename_map = {}

if "start" in ti.columns: rename_map["start"] = "start_time"
if "begin" in ti.columns: rename_map["begin"] = "start_time"
if "start_time" in ti.columns: rename_map["start_time"] = "start_time"

if "end" in ti.columns: rename_map["end"] = "end_time"
if "finish" in ti.columns: rename_map["finish"] = "end_time"
if "end_time" in ti.columns: rename_map["end_time"] = "end_time"

if "type" in ti.columns: rename_map["type"] = "event_type"
if "event" in ti.columns: rename_map["event"] = "event_type"
if "event_type" in ti.columns: rename_map["event_type"] = "event_type"

ti = ti.rename(columns=rename_map)

# Now ensure required columns exist
expected = {"start_time", "end_time", "event_type"}
missing = expected - set(ti.columns)
if missing:
    raise ValueError(f"interval.csv missing columns: {missing}")

ti.to_sql("TargetInterval", conn, if_exists="append", index=False)

conn.commit()


In [ ]:
pd.read_sql_query("""
SELECT r.name, MAX(s.x_coord) AS max_x, MIN(s.x_coord) AS min_x
FROM Robot r
JOIN SensorReading s ON r.robot_id = s.robot_id
GROUP BY r.robot_id, r.name;
""", conn)


In [ ]:
pd.read_sql_query("""
SELECT 
    r.name,
    MAX(s.y_coord) AS max_y,
    MIN(s.y_coord) AS min_y
FROM Robot r
JOIN SensorReading s ON r.robot_id = s.robot_id
GROUP BY r.robot_id, r.name;

""", conn)

In [ ]:
import sqlite3
import pandas as pd

# Reconnect to the database
conn = sqlite3.connect("robot.db")

# ---------- Task 1: regions where Astro and IamHuman are close ----------
task1_query = """
SELECT
    CASE 
        WHEN a.x_coord < h.x_coord THEN a.x_coord 
        ELSE h.x_coord 
    END AS x_min,

    CASE 
        WHEN a.x_coord > h.x_coord THEN a.x_coord 
        ELSE h.x_coord 
    END AS x_max,

    CASE 
        WHEN a.y_coord < h.y_coord THEN a.y_coord 
        ELSE h.y_coord 
    END AS y_min,

    CASE 
        WHEN a.y_coord > h.y_coord THEN a.y_coord 
        ELSE h.y_coord 
    END AS y_max,

    a.timestamp
FROM SensorReading a
JOIN Robot r1 
    ON a.robot_id = r1.robot_id AND r1.name = 'Astro'
JOIN SensorReading h 
    ON a.timestamp = h.timestamp
JOIN Robot r2 
    ON h.robot_id = r2.robot_id AND r2.name = 'IamHuman'
WHERE ABS(a.x_coord - h.x_coord) < 1.0
  AND ABS(a.y_coord - h.y_coord) < 1.0;
"""

print("=== Task 1: Regions where Astro and IamHuman are close ===")
display(pd.read_sql_query(task1_query, conn))


# ---------- Task 2: how many seconds they are close ----------
task2_query = """
SELECT COUNT(*) AS seconds_close
FROM SensorReading a
JOIN Robot r1 
    ON a.robot_id = r1.robot_id AND r1.name = 'Astro'
JOIN SensorReading h 
    ON a.timestamp = h.timestamp
JOIN Robot r2 
    ON h.robot_id = r2.robot_id AND r2.name = 'IamHuman'
WHERE ABS(a.x_coord - h.x_coord) < 1.0
  AND ABS(a.y_coord - h.y_coord) < 1.0;
"""

print("=== Task 2: Number of seconds close ===")
display(pd.read_sql_query(task2_query, conn))



In [ ]:
query = """
WITH movement AS (
    SELECT
        robot_id,
        timestamp,
        x_coord,
        y_coord,
        LAG(x_coord) OVER (PARTITION BY robot_id ORDER BY timestamp) AS prev_x,
        LAG(y_coord) OVER (PARTITION BY robot_id ORDER BY timestamp) AS prev_y
    FROM SensorReading
),
dist AS (
    SELECT
        robot_id,
        timestamp,
        CASE WHEN prev_x IS NULL THEN 0
             ELSE sqrt((x_coord - prev_x)*(x_coord - prev_x) +
                       (y_coord - prev_y)*(y_coord - prev_y))
        END AS step_dist
    FROM movement
)
SELECT
    t.interval_id,
    CASE WHEN (SUM(d.step_dist) / (t.end_time - t.start_time)) < 0.2
         THEN 'Yes' ELSE 'No'
    END AS speed_below_threshold
FROM TargetInterval t
LEFT JOIN dist d
    ON d.timestamp BETWEEN t.start_time AND t.end_time
GROUP BY t.interval_id;
"""

pd.read_sql_query(query, conn)
